In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers mlflow torch torchvision timm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import mlflow

MLFLOW_DIR = "/content/drive/MyDrive/deepfake-detection/runs"
mlflow.set_tracking_uri(f"file:{MLFLOW_DIR}")

In [4]:
import os
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix, recall_score, precision_score
import numpy as np
from PIL import Image
import random
import timm
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import ViTFeatureExtractor, ViTForImageClassification

# MLFlow Setup


In [5]:
config = {
    "experiment_name": "better_ViT_deepfake_multiclass_large_random",
    "model_name": "vit_base_patch16_224",
    "data_path": "/content/drive/MyDrive/deepfake-detection/datasets/large_dataset_random",
    "batch_size": 16,
    "num_epochs": 10,
    "learning_rate": 2e-5,
    "img_height": 224,
    "img_width": 224,
    "optimizer": "adamw",
    "loss_fn": "crossentropy",
    "random_seed": 42
}

In [6]:
mlflow.set_tracking_uri("file:/content/drive/MyDrive/deepfake-detection/runs")
mlflow.set_experiment(config["experiment_name"])

2025/06/11 23:01:28 INFO mlflow.tracking.fluent: Experiment with name 'better_ViT_deepfake_multiclass_large_random' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/drive/MyDrive/deepfake-detection/runs/275078373199444901', creation_time=1749682888967, experiment_id='275078373199444901', last_update_time=1749682888967, lifecycle_stage='active', name='better_ViT_deepfake_multiclass_large_random', tags={}>

# Data Preparation

In [7]:
import os
import random
from PIL import Image
from torch.utils.data import Dataset

class MultiClassDataset(Dataset):
    """
    Multi-class dataset. Optionally, for the "real" class, uses only a fraction of the images.
    """
    def __init__(self, root_dir, transform=None, real_fraction=0.2, real_class_name="real", seed=42):
        self.samples = []
        self.transform = transform
        self.class_to_idx = {}
        self.idx_to_class = {}

        subdirs = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])

        for idx, subdir in enumerate(subdirs):
            self.class_to_idx[subdir] = idx
            self.idx_to_class[idx] = subdir

        rng = random.Random(seed)

        for subdir in subdirs:
            subdir_path = os.path.join(root_dir, subdir)
            label = self.class_to_idx[subdir]
            images = [
                os.path.join(subdir_path, fname)
                for fname in os.listdir(subdir_path)
                if fname.lower().endswith(('.jpg', '.jpeg', '.png'))
            ]
            # If this is the "real" class, optionally subsample
            if subdir.lower() == real_class_name.lower() and real_fraction < 1.0:
                n_keep = max(1, int(len(images) * real_fraction))
                images = rng.sample(images, n_keep)
            self.samples.extend((img_path, label) for img_path in images)

        self.num_classes = len(self.class_to_idx)
        self.total_images = len(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, int(label)


In [8]:
# Use HuggingFace feature extractor for ViT
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")

def extractor_transform(img):
    # Returns a Tensor [3,224,224]
    return feature_extractor(images=img, return_tensors="pt")['pixel_values'].squeeze(0)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [9]:
torch.manual_seed(config["random_seed"])

transform = transforms.Compose([
    transforms.Resize((config["img_height"], config["img_width"])),
    transforms.ToTensor()
])

dataset = MultiClassDataset(config["data_path"], transform=transform)

total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(config["random_seed"])
)

train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, num_workers=2)

# Model Definition

In [10]:
def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    all_labels, all_outputs = [], []
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.long)  # <-- keep as long/int64!
        optimizer.zero_grad()
        outputs = model(images)
        logits = outputs.logits
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        all_labels.extend(labels.cpu().numpy())
        all_outputs.extend(logits.detach().cpu().numpy())  # logits; if you want probabilities, apply softmax

    acc = correct / total
    avg_loss = running_loss / total
    return avg_loss, acc, np.array(all_labels), np.array(all_outputs)


In [11]:
def evaluate(model, loader, loss_fn, device):
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    all_labels, all_outputs = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device, dtype=torch.long)
            outputs = model(images)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            running_loss += loss.item() * images.size(0)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(logits.cpu().numpy())
    acc = correct / total
    avg_loss = running_loss / total
    return avg_loss, acc, np.array(all_labels), np.array(all_outputs)

In [12]:
def plot_and_log_curve(train_values, val_values, ylabel, fname):
    import matplotlib.pyplot as plt
    plt.figure()
    plt.plot(train_values, label='Train')
    plt.plot(val_values, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.title(f'{ylabel} Curve')
    plt.legend()
    plt.grid(True)
    plt.savefig(fname)
    mlflow.log_artifact(fname)
    plt.close()

In [13]:
def plot_and_log_confusion_matrix(y_true, y_pred, step, label="val"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title(f"{label.capitalize()} Confusion Matrix")
    fname = f"{label}_confusion_matrix_{step}.png"
    plt.savefig(fname)
    mlflow.log_artifact(fname)
    plt.close()

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = dataset.num_classes
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(device)

loss_fn = nn.CrossEntropyLoss()

if config["optimizer"].lower() == "adagrad":
    optimizer = optim.Adagrad(model.parameters(), lr=config["learning_rate"])
else:
    optimizer = optim.AdamW(model.parameters(), lr=config["learning_rate"])

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import os
import shutil
import numpy as np
import warnings
from sklearn.exceptions import UndefinedMetricWarning


best_val_f1 = 0
best_model_path = "/tmp/best_vit.pth"

with mlflow.start_run():
    mlflow.log_params(config)

    train_loss_list, train_acc_list = [], []
    val_loss_list, val_acc_list = [], []

    for epoch in range(config["num_epochs"]):
        train_loss, train_acc, train_labels, train_outputs = train_one_epoch(
            model, train_loader, optimizer, loss_fn, device)
        val_loss, val_acc, val_labels, val_outputs = evaluate(
            model, val_loader, loss_fn, device)
        val_labels = np.array(val_labels).astype(int).flatten()

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)

        val_preds = np.argmax(val_outputs, axis=1)
        val_preds = np.array(val_preds).astype(int).flatten()

        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UndefinedMetricWarning)
            if len(np.unique(val_labels)) < 2:
                val_f1 = 0.0
                val_tpr = 0.0
                val_fpr = 0.0
                val_auc = float('nan')
            else:
                val_f1 = f1_score(val_labels, val_preds, average="macro", zero_division=0)
                val_tpr = recall_score(val_labels, val_preds, average="macro", zero_division=0)
                val_fpr = 1 - precision_score(val_labels, val_preds, average="macro", zero_division=0)
                try:
                    val_auc = roc_auc_score(val_labels, val_outputs)
                except:
                    val_auc = float('nan')

        mlflow.log_metrics({
            "train_loss": train_loss, "train_acc": train_acc,
            "val_loss": val_loss, "val_acc": val_acc,
            "val_f1": val_f1, "val_tpr": val_tpr, "val_auc": val_auc,
        }, step=epoch)

        print(f"Epoch {epoch+1}/{config['num_epochs']}: "
              f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
              f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}, "
              f"val_f1={val_f1:.4f}, val_auc={val_auc:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), best_model_path)
            mlflow.log_artifact(best_model_path)

    plot_and_log_curve(train_loss_list, val_loss_list, "Loss", "loss_curve.png")
    plot_and_log_curve(train_acc_list, val_acc_list, "Accuracy", "accuracy_curve.png")

    # Save the full model for reproducibility (architecture + weights)
    model.save_pretrained("/tmp/final_vit_model")
    feature_extractor.save_pretrained("/tmp/final_vit_extractor")
    mlflow.log_artifacts("/tmp/final_vit_model", artifact_path="final_model")
    mlflow.log_artifacts("/tmp/final_vit_extractor", artifact_path="feature_extractor")
    shutil.rmtree("/tmp/final_vit_model")
    shutil.rmtree("/tmp/final_vit_extractor")

    print("Training finished. Best validation F1:", best_val_f1)
    plot_and_log_confusion_matrix(val_labels, val_preds, step="final", label="val")


Epoch 1/10: train_loss=0.8599, train_acc=0.7240, val_loss=0.5365, val_acc=0.8093, val_f1=0.8052, val_auc=nan
Epoch 2/10: train_loss=0.3658, train_acc=0.8926, val_loss=0.3146, val_acc=0.9027, val_f1=0.9036, val_auc=nan
Epoch 3/10: train_loss=0.2294, train_acc=0.9360, val_loss=0.2659, val_acc=0.9000, val_f1=0.9028, val_auc=nan
Epoch 4/10: train_loss=0.1636, train_acc=0.9569, val_loss=0.2544, val_acc=0.9027, val_f1=0.9031, val_auc=nan
Epoch 5/10: train_loss=0.1160, train_acc=0.9737, val_loss=0.2816, val_acc=0.8960, val_f1=0.8996, val_auc=nan
Epoch 6/10: train_loss=0.0951, train_acc=0.9760, val_loss=0.2217, val_acc=0.9187, val_f1=0.9206, val_auc=nan
Epoch 7/10: train_loss=0.0528, train_acc=0.9923, val_loss=0.3636, val_acc=0.8760, val_f1=0.8820, val_auc=nan
Epoch 8/10: train_loss=0.0454, train_acc=0.9917, val_loss=0.2383, val_acc=0.9240, val_f1=0.9248, val_auc=nan
Epoch 9/10: train_loss=0.0355, train_acc=0.9943, val_loss=0.2441, val_acc=0.9200, val_f1=0.9210, val_auc=nan
Epoch 10/10: train_

In [16]:
model.load_state_dict(torch.load(best_model_path))
model.eval()

test_labels, test_outputs = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        logits = outputs.logits
        labels = labels.to(device)  # <--- ADD THIS LINE
        loss = loss_fn(logits, labels)
        test_outputs.extend(logits.cpu().numpy())  # Logits, shape [batch, num_classes]
        test_labels.extend(labels.cpu().numpy())    # Integers, shape [batch]

test_labels = np.array(test_labels).astype(int)  # Shape: [N,]
test_outputs = np.array(test_outputs)            # Shape: [N, num_classes]
test_preds = np.argmax(test_outputs, axis=1)     # Shape: [N,]


In [17]:
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score

# test_labels: shape (N,) integers; test_preds: shape (N,) integers; test_outputs: shape (N, num_classes) logits or probs

test_f1 = f1_score(test_labels, test_preds, average="macro", zero_division=0)
test_recall = recall_score(test_labels, test_preds, average="macro", zero_division=0)
test_precision = precision_score(test_labels, test_preds, average="macro", zero_division=0)

# For multiclass AUC, one-hot encode labels
import numpy as np
num_classes = np.max(test_labels) + 1
test_labels_onehot = np.eye(num_classes)[test_labels]

try:
    test_auc = roc_auc_score(test_labels_onehot, np.array(test_outputs), multi_class="ovr")
except Exception as e:
    test_auc = float('nan')

print(f"Test F1: {test_f1:.4f}, Test Recall: {test_recall:.4f}, Test Precision: {test_precision:.4f}, Test AUC: {test_auc:.4f}")


Test F1: 0.9293, Test Recall: 0.9289, Test Precision: 0.9300, Test AUC: 0.9908


In [18]:
mlflow.log_metrics({
    "test_f1": test_f1,
    "test_recall": test_recall,
    "test_precision": test_precision,
    "test_auc": test_auc
})


plot_and_log_confusion_matrix(test_labels, test_preds, step="final", label="test")

In [19]:
# Get the classes and their values from the model
classes = dataset.idx_to_class
class_values = list(classes.values())
classes, class_values

({0: 'real',
  1: 'stable_diffusion_xl',
  2: 'stylegan1',
  3: 'stylegan2',
  4: 'thispersondoesnotexist'},
 ['real',
  'stable_diffusion_xl',
  'stylegan1',
  'stylegan2',
  'thispersondoesnotexist'])